# Revenue Forecasting
This notebook is used as an analytical platform to demonstrate price forecasting of future sales for 50 items across 10 stores. The approaches used vary in complexity to explore different approaches and compare the results.

No approach in this notebook is carried out to the fullest extent that should be applied for actual business analytics.

# Packages

In [ ]:
# Base
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Modeling
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 1. Loading The Data
The data used herein has been obtained through Kaggle's ["Store Item Demand Forecasting Challenge"](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data). This data spans 5-years of sales data and will be used to predict future sales. The source of the data is not provided.

In [ ]:
# Loading the data
train = pd.read_csv('./data/train.csv', parse_dates=['date'], index_col=['date'])
test = pd.read_csv('./data/test.csv', parse_dates=['date'], index_col=['date'])

df = train.copy()

# Display basic information
print(train.shape, test.shape)
df.head()

# 2. Preprocessing & Feature Engineering
Using the information already given, we can expand fields to further look at trends such as sales by day of the week or month.

In [ ]:
# General descriptors
print("Date Range:", train.index.min(), " to ", train.index.max())

sales_avg = train.sales.mean()
print("Average Item Sales:", sales_avg)

# Feature Engineering - Expand date information
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofweek'] = df.index.dayofweek
df['quarter'] = ((df['month']+2)/3).astype(int)

df.head(n=100)

# 3. Analyzing the Data
Understanding the data is one of the most important things a data analyst can do. Looking for trends in the data may give insight on how to approach finding the answer you are looking for. Better yet, it could give you the answer you're looking for!

In [ ]:
# Histogram of sales
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
for i in range(1,11):
    if i < 6:
        # Top Row
        train[train.store == i].sales.hist(ax=axes[0, i-1])
        axes[0, i-1].set_title("Store " + str(i), fontsize = 15)
    else:
        # Bottom Row
        train[train.store == i].sales.hist(ax=axes[1, i - 6])
        axes[1, i-6].set_title("Store " + str(i), fontsize = 15)
plt.tight_layout(pad=3)
plt.suptitle("Item Sales by Store");

The histograms of item sales per store are nearly identical and suggest to me that this is either a synthetic dataset or an extreme relationship exists.

##  Relations
There are many relations that can be explored:
 - item-store relationship
 - seasonal (or quarter) relationship
 - days of the week relationship
 - changes by year or month

Each could provide different and unique insights into the trends we are looking to explore. Visualizing this data may even extrapolate the trend right here and now to give us a quick answer. However, Only a few of these relationships will be demonstrated as a proof of concept. The rest is left to the reader as an exercise.

### Item-Store relationship

In [ ]:
table_item_store = pd.pivot_table(df, index='store', columns='item',
                                values='sales', aggfunc=np.mean).values
# print(item_store_table)

plt.figure(figsize=(15, 5))

# Items
plt.subplot(122)
plt.plot(table_item_store / table_item_store.mean(0)[np.newaxis])
plt.title("Items")
plt.xlabel("Store")
plt.ylabel("Sales Relative to Average")

# Stores
plt.subplot(131)
plt.plot(table_item_store.T / table_item_store.T.mean(0)[np.newaxis])
plt.title("Stores")
plt.xlabel("Item")
plt.ylabel("Sales Relative to Average")
plt.legend(labels=df['store'])

plt.show()

### Item-Date relationships

In [ ]:
table_item_dow = pd.pivot_table(df, index='dayofweek', columns='item',
                                values='sales', aggfunc=np.mean).values
table_item_year = pd.pivot_table(df, index='year', columns='item',
                               values='sales', aggfunc=np.mean).values

plt.figure(figsize=(15, 5))

# Day of the week
week_days = ['', 'Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
ax1 = plt.subplot(121)
ax1.set_xticklabels(week_days)
plt.plot(table_item_dow / table_item_dow.mean(0)[np.newaxis])
plt.title("Days of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Sales Relative to Average")

# Years
years = np.arange(2012.5,2018.5, 0.5)
ax2 = plt.subplot(122)
ax2.set_xticklabels(years)
plt.plot(table_item_year / table_item_year.mean(0)[np.newaxis])
plt.title("Years")
plt.xlabel("Year")
plt.ylabel("Sales Relative to Average")

plt.show()

It is clear to see by the graphs that items and stores are performing without degeneracies in the data.

*note*: The warning produced by the graphs is due to a [bug.](https://github.com/pandas-dev/pandas/issues/35684) However, in order to provide more clarity, I choose to keep the warning and display the x labels more accurately.

# 4. Simple Linear Regression
Using an approach least-squares polynomial fit to predict future yearly sales.

In [ ]:
year_table = pd.pivot_table(df, index='year', values='sales', aggfunc=np.mean)
year_table = year_table / sales_avg

years = np.arange(2013, 2019)
annual_sales_avg = year_table.values.squeeze()

# Perform polynomial fits
p1 = np.poly1d(np.polyfit(years[:-1], annual_sales_avg, 1))
p2 = np.poly1d(np.polyfit(years[:-1], annual_sales_avg, 2))

print("Relative Sales in 2018 based on Degree-1 (Linear) Fit:  ", "{:.4f}".format(p1(2018)))
print("Relative Sales in 2018 based on Degree-2 (Quadratic) Fit:", "{:.4f}".format(p2(2018)))

In [ ]:
# Plot the results
plt.figure(figsize=(8,6))

plt.plot(years[:-1], annual_sales_avg, 'o')
plt.plot(years, p1(years), 'C0')
plt.plot(years, p2(years), 'C1')

plt.title("Relative Sales by Year")
plt.legend(["Annual Sales", "Linear fit", "Quadratic fit"])
plt.xlim(2012.5, 2018.5)
plt.ylabel("Relative Sales")
plt.xlabel("Year")

plt.show()

Note that the quadratic fit more-closely follows the trend. Hence, the quadratic fit would be more accurate over the linear fit in the prediction of the 2018 season.

# 5. Advanced Prediction - LGBM
Light Gradient boosting is high-performance gradient boosting for machine learning tasks dealing with either large or small data-sets. Here, we are going to train a model based on sales data in an attempt to better predict future sales.

## Data Partitioning
Start by splitting and formatting the training and testing sets.

*note*: Test set is interchangably used with validation set.

In [ ]:
# Sort by data for time series split
df = df.sort_index()

# Train-Validation Split
train = df.loc[(df.index < "2017-10-01"), :]
val = df.loc[(df.index >= "2017-10-01") & (df.index < "2018-01-01"), :]

train_features = [col for col in train.columns if col not in ['sales', 'year']]

X_train = train[train_features]
Y_train = train['sales']
X_val = val[train_features]
Y_val = val['sales']

train_dataset = lgb.Dataset(X_train, Y_train)
test_dataset = lgb.Dataset(X_val, Y_val)

print(Y_train.shape, X_train.shape, Y_val.shape, X_val.shape)

After we have our training and validation sets, we will configure LGBM to train the model. 

Explaining every detail of this process is a little out of the scope of this simplified approach. One could change the loss function, the number of iterations, the learning rate, among the myriad of parameters.

In [ ]:
# MAE function
def mean_absolute_error(preds, data):
    actuals = data.get_label()
    err = (actuals - preds).sum()
    is_higher_better = False
    return "MAE", err, is_higher_better


# LGBM Configuration
params = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.7,
    "force_row_wise": True,
}

rounds = 15

# Train the model
booster = lgb.train(params,
                    feval=mean_absolute_error,
                    train_set=train_dataset,
                    valid_sets=(test_dataset,),
                    num_boost_round=rounds
                    )

## Computing Error Rate
It is important to see how the models performed on the training & test sets. Ideally, the model should get 100% accuracy on the training set and a very high accuracy on the validation set.

In [ ]:
# Compute accuracy of our model
test_preds = booster.predict(X_val)
train_preds = booster.predict(X_train)

print("\nTrain R2 Score :", "{:.2f}".format(r2_score(Y_train, train_preds)))
print("Test  R2 Score :", "{:.2f}".format(r2_score(Y_val, test_preds)))

In [ ]:
pd.DataFrame(train_preds).describe([0.1, 0.75, 0.8, 0.9, 0.95, 0.99]).T

## Figuring Out What Parameters Mattered
In real life, we want to see which parameters are having the biggest impact on the value we are trying to predict.

In [ ]:
# Get feature importance
lgb.plot_importance(booster, importance_type="gain")

# Conclusion
While it would be nice to use one-size-fits-all algorithms on data, it is necessary to understand the data that is being interpreted. A guess that this is synthetic data would have us using different methods than real-world data and even then, the nature of that data would each require unique solutions.